## MODEL TERBARU PENYAKIT IKAN

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Menetapkan path untuk data training, validasi, dan testing
train_dir = 'dataset_bersama/Train/'
validation_dir = 'dataset_bersama/Validation/'
test_dir = 'dataset_bersama/Test/'

# Menggunakan ImageDataGenerator untuk preprocessing dan augmentasi data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)  # Hanya rescale untuk validasi dan testing

# Menggunakan flow_from_directory untuk memuat gambar
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

# Memastikan jumlah kelas berdasarkan generator
num_classes = len(train_generator.class_indices)

# Membangun model CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Sesuaikan jumlah kelas sesuai dengan dataset
])

# Kompilasi model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Melatih model
history = model.fit(
      train_generator,
      steps_per_epoch=100,  # Sesuaikan dengan jumlah data Anda
      epochs=25,
      validation_data=validation_generator,
      validation_steps=50)  # Sesuaikan dengan jumlah data validasi Anda

# Evaluasi model dengan data testing
model.evaluate(test_generator)

Found 1147 images belonging to 5 classes.
Found 100 images belonging to 5 classes.
Found 497 images belonging to 5 classes.


C:\Users\Rhaayn\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/25


C:\Users\Rhaayn\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 36/100 ━━━━━━━━━━━━━━━━━━━━ 58s 914ms/step - accuracy: 0.2451 - loss: 2.0010 

c:\Users\Rhaayn\anaconda3\envs\mypython2\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 43s 342ms/step - accuracy: 0.2434 - loss: 1.8005 - val_accuracy: 0.2100 - val_loss: 1.3223
Epoch 2/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 26s 234ms/step - accuracy: 0.3267 - loss: 1.4766 - val_accuracy: 0.1900 - val_loss: 1.2971
Epoch 3/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 25s 230ms/step - accuracy: 0.3379 - loss: 1.4551 - val_accuracy: 0.2100 - val_loss: 1.3525
Epoch 4/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 26s 232ms/step - accuracy: 0.4000 - loss: 1.3711 - val_accuracy: 0.2000 - val_loss: 1.5059
Epoch 5/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 25s 227ms/step - accuracy: 0.4226 - loss: 1.3265 - val_accuracy: 0.1500 - val_loss: 1.4500
Epoch 6/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 26s 229ms/step - accuracy: 0.4730 - loss: 1.2238 - val_accuracy: 0.1500 - val_loss: 1.3174
Epoch 7/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 25s 224ms/step - accuracy: 0.5150 - loss: 1.1280 - val_accuracy: 0.2500 - val_loss: 1.2518
Epoch 8/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 24s 220ms/step - accuracy: 0.5324 - loss: 1.1496 - val

[0.5028824210166931, 0.8390341997146606]

## MENYIMPAN MODEL

In [8]:
model.save('model_cnn.h5')

## PENGUJIAN

In [10]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

model = load_model('model_cnn.h5')

# Fungsi untuk memuat dan mempersiapkan gambar
def load_and_prepare_image(image_path):
    # Memuat gambar
    img = image.load_img(image_path, target_size=(150, 150))
    # Mengubah gambar menjadi numpy array
    img_array = image.img_to_array(img)
    # Normalisasi gambar
    img_array /= 255
    # Expand dimensi gambar agar cocok dengan input model (menambahkan batch size dimensi)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Mendapatkan pemetaan dari class indices
class_indices = train_generator.class_indices
# Membalik dictionary untuk mendapatkan mapping dari indeks ke label
label_map = {v: k for k, v in class_indices.items()}

# Path ke gambar baru
new_image_path = 'ujini.png'

# Memuat dan mempersiapkan gambar
new_image = load_and_prepare_image(new_image_path)

# Melakukan prediksi
predictions = model.predict(new_image)
predicted_class_index = np.argmax(predictions, axis=1)[0]  # Mengambil indeks dengan probabilitas tertinggi

# Mendapatkan label kelas dari indeks
predicted_class_label = label_map[predicted_class_index]

# Output label prediksi
print("Label prediksi:", predicted_class_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
Label prediksi: Tail and Fin Rot
